In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm

from utils import Handler

In [2]:
def four_point_transform(image, pts):

    rect = np.array((pts[0], pts[1], pts[3], pts[2]))
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # constract the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth, 0],
        [0, maxHeight],
        [maxWidth, maxHeight]], dtype="float32")
    # dst = np.array([
    #     [int(maxWidth/3), int(maxHeight/3)],
    #     [int(maxWidth*2/3), int(maxHeight/3)],
    #     [int(maxWidth/2), int(maxHeight/2)],
    #     [int(maxWidth/2), maxHeight]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    # M = cv2.getPerspectiveTransform(rect, dst)
    h, mask = cv2.findHomography(pts, dst, cv2.RANSAC)
    height, width, channels = image.shape
    warped = cv2.warpPerspective(image, h, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [3]:
handler = Handler('/home/dl/1_study/0_BSU/master_thesis/repos/insightface/alignment/\
coordinateReg/model/2d106det', 0, ctx_id=0, det_size=640)

loading /home/dl/1_study/0_BSU/master_thesis/repos/insightface/alignment/coordinateReg/model/2d106det 0
[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [4]:
path = "/home/dl/1_study/0_BSU/master_thesis/dataset/emotions/base/"
dataset_1 = "fer2013" 
dataset_2 = "jaffedbase"
dataset_3 = "KDEF_and_AKDEF"
dataset_4 = "AffectNet-8Labels"
color = (200, 160, 75)

In [5]:
def test_points(points, shape):
    checked_points = []
    for point in points:
        checked_point = point
        
        if point[0] < 0:
            checked_point[0] = 1
        if point[0] > shape[1]:
            checked_point[0] = shape[1] - 1
            
        if point[1] < 0:
            checked_point[1] = 1
        if point[1] > shape[0]:
            checked_point[1] = shape[0] - 1
            
        checked_points.append(checked_point)
        
    return checked_points

In [6]:
def crop_and_dewarp():
    pass

# fer2013

In [9]:
folder_train = "train"
folder_public_test = "public_test"
folder_private_test = "private_test"

In [23]:
for folder_name in [folder_train, folder_public_test, folder_private_test]:
    for emotion in os.listdir(os.path.join(path, dataset_1, "base", folder_name)):
        for img_name in tqdm(os.listdir(os.path.join(path, dataset_1, "base", folder_name, emotion))):
#             print(os.path.join(emotion, img_name))
            img = cv2.imread(os.path.join(path, dataset_1, "base", folder_name, emotion, img_name))
            tim = img.copy()
            
            preds = handler.get(img, get_all=True)
            if preds == []:
                print("--------SKIP--------")
                continue
            else:
                preds = preds[0]
            
            # crop stage
            x1, y1 = int(np.min(preds[:, 0])), int(np.min(preds[:, 1]))
            x2, y2 = int(np.max(preds[:, 0])), int(np.min(preds[:, 1]))
            x3, y3 = int(np.min(preds[:, 0])), int(np.max(preds[:, 1]))
            x4, y4 = int(np.max(preds[:, 0])), int(np.max(preds[:, 1]))
            
            points = test_points([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], img.shape)
            
            x1, y1 = points[0]
            x2, y2 = points[1]
            x3, y3 = points[2]
            x4, y4 = points[3]
            
            cv2.circle(tim, (x1, y1), 1, (256, 0, 0), 2, cv2.LINE_AA)
            cv2.circle(tim, (x2, y2), 1, (256, 0, 0), 2, cv2.LINE_AA)
            cv2.circle(tim, (x3, y3), 1, (256, 0, 0), 2, cv2.LINE_AA)
            cv2.circle(tim, (x4, y4), 1, (256, 0, 0), 2, cv2.LINE_AA)
#             print("Pred")
            
            crop_img = img[min(y1, y2):max(y3, y4), min(x1, x3):max(x2, x4)]
            cv2.imwrite(os.path.join(path, dataset_1, "preprocess", "crop", folder_name,
                                     emotion, img_name), 
                        crop_img)
            
            # dewarp stage
            x1, y1 = preds[48].astype(np.int)
            x2, y2 = preds[105].astype(np.int)
            x3, y3 = preds[5].astype(np.int)
            x4, y4 = preds[21].astype(np.int)
            
            points = test_points([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], img.shape)
            
            x1, y1 = points[0]
            x2, y2 = points[1]
            x3, y3 = points[2]
            x4, y4 = points[3]
            
            cv2.circle(tim, (x1, y1), 1, (0, 0, 256), 2, cv2.LINE_AA)
            cv2.circle(tim, (x2, y2), 1, (0, 0, 256), 2, cv2.LINE_AA)
            cv2.circle(tim, (x3, y3), 1, (0, 0, 256), 2, cv2.LINE_AA)
            cv2.circle(tim, (x4, y4), 1, (0, 0, 256), 2, cv2.LINE_AA)
#             print("Pred")
            
            dewarp_img = four_point_transform(img,np.array([[x1, y1], 
                                                            [x2, y2], 
                                                            [x3, y3], 
                                                            [x4, y4]]))
            cv2.imwrite(os.path.join(path, dataset_1, "preprocess", "dewarp", folder_name,
                                     emotion, img_name), 
                        dewarp_img)
            
            
            # for pred in preds:
            pred = np.round(preds).astype(np.int)
            for i in range(pred.shape[0]):
                p = tuple(pred[i])
                cv2.circle(tim, p, 1, color, 1, cv2.LINE_AA)
            cv2.imwrite(os.path.join(path, dataset_1, "preprocess", "landmarks", folder_name,
                                     emotion, img_name), 
                        tim)
            

  1%|          | 53/7215 [00:00<02:15, 53.00it/s]

--------SKIP--------


  1%|          | 71/7215 [00:01<02:10, 54.70it/s]

--------SKIP--------


  2%|▏         | 132/7215 [00:02<02:04, 57.07it/s]

...

 87%|████████▋ | 425/491 [00:07<00:01, 57.65it/s]

--------SKIP--------
--------SKIP--------


 93%|█████████▎| 455/491 [00:07<00:00, 56.85it/s]

--------SKIP--------


100%|██████████| 491/491 [00:08<00:00, 56.88it/s]


# jaffedbase/KDEF

In [27]:
for emotion in os.listdir(os.path.join(path, dataset_3, "base")):
    for img_name in tqdm(os.listdir(os.path.join(path, dataset_3, "base", emotion))):
#             print(os.path.join(emotion, img_name))
        img = cv2.imread(os.path.join(path, dataset_3, "base", emotion, img_name))
        tim = img.copy()

        preds = handler.get(img, get_all=True)
        if preds == []:
            print("--------SKIP--------")
            continue
        else:
            preds = preds[0]

        # crop stage
        x1, y1 = int(np.min(preds[:, 0])), int(np.min(preds[:, 1]))
        x2, y2 = int(np.max(preds[:, 0])), int(np.min(preds[:, 1]))
        x3, y3 = int(np.min(preds[:, 0])), int(np.max(preds[:, 1]))
        x4, y4 = int(np.max(preds[:, 0])), int(np.max(preds[:, 1]))

        points = test_points([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], img.shape)

        x1, y1 = points[0]
        x2, y2 = points[1]
        x3, y3 = points[2]
        x4, y4 = points[3]

        cv2.circle(tim, (x1, y1), 1, (256, 0, 0), 2, cv2.LINE_AA)
        cv2.circle(tim, (x2, y2), 1, (256, 0, 0), 2, cv2.LINE_AA)
        cv2.circle(tim, (x3, y3), 1, (256, 0, 0), 2, cv2.LINE_AA)
        cv2.circle(tim, (x4, y4), 1, (256, 0, 0), 2, cv2.LINE_AA)
#             print("Pred")

        crop_img = img[min(y1, y2):max(y3, y4), min(x1, x3):max(x2, x4)]
        cv2.imwrite(os.path.join(path, dataset_3, "preprocess", "crop",
                                 emotion, img_name), 
                    crop_img)

        # dewarp stage
        x1, y1 = preds[48].astype(np.int)
        x2, y2 = preds[105].astype(np.int)
        x3, y3 = preds[5].astype(np.int)
        x4, y4 = preds[21].astype(np.int)

        points = test_points([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], img.shape)

        x1, y1 = points[0]
        x2, y2 = points[1]
        x3, y3 = points[2]
        x4, y4 = points[3]

        cv2.circle(tim, (x1, y1), 1, (0, 0, 256), 2, cv2.LINE_AA)
        cv2.circle(tim, (x2, y2), 1, (0, 0, 256), 2, cv2.LINE_AA)
        cv2.circle(tim, (x3, y3), 1, (0, 0, 256), 2, cv2.LINE_AA)
        cv2.circle(tim, (x4, y4), 1, (0, 0, 256), 2, cv2.LINE_AA)
#             print("Pred")

        dewarp_img = four_point_transform(img,np.array([[x1, y1], 
                                                        [x2, y2], 
                                                        [x3, y3], 
                                                        [x4, y4]]))
        cv2.imwrite(os.path.join(path, dataset_3, "preprocess", "dewarp",
                                 emotion, img_name), 
                    dewarp_img)


        # for pred in preds:
        pred = np.round(preds).astype(np.int)
        for i in range(pred.shape[0]):
            p = tuple(pred[i])
            cv2.circle(tim, p, 1, color, 1, cv2.LINE_AA)
        cv2.imwrite(os.path.join(path, dataset_3, "preprocess", "landmarks",
                                 emotion, img_name), 
                    tim)


 14%|█▍        | 100/697 [00:03<00:21, 28.10it/s]

--------SKIP--------


 18%|█▊        | 128/697 [00:04<00:19, 28.63it/s]

--------SKIP--------


 67%|██████▋   | 465/697 [00:17<00:08, 28.27it/s]

--------SKIP--------


100%|██████████| 700/700 [00:26<00:00, 26.50it/s]


# affectnet-8labels

In [7]:
folder_train = "train"
folder_val = "val"

In [8]:
for folder_name in [folder_train, folder_val]:
    for emotion in os.listdir(os.path.join(path, dataset_4, "base", folder_name)):
        for img_name in tqdm(os.listdir(os.path.join(path, dataset_4, "base", folder_name, emotion))):
#             print(os.path.join(emotion, img_name))
            img = cv2.imread(os.path.join(path, dataset_4, "base", folder_name, emotion, img_name))
            tim = img.copy()
            
            preds = handler.get(img, get_all=True)
            if preds == []:
                print("--------SKIP--------")
                continue
            else:
                preds = preds[0]
            
            # crop stage
            x1, y1 = int(np.min(preds[:, 0])), int(np.min(preds[:, 1]))
            x2, y2 = int(np.max(preds[:, 0])), int(np.min(preds[:, 1]))
            x3, y3 = int(np.min(preds[:, 0])), int(np.max(preds[:, 1]))
            x4, y4 = int(np.max(preds[:, 0])), int(np.max(preds[:, 1]))
            
            points = test_points([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], img.shape)
            
            x1, y1 = points[0]
            x2, y2 = points[1]
            x3, y3 = points[2]
            x4, y4 = points[3]
            
            cv2.circle(tim, (x1, y1), 1, (256, 0, 0), 2, cv2.LINE_AA)
            cv2.circle(tim, (x2, y2), 1, (256, 0, 0), 2, cv2.LINE_AA)
            cv2.circle(tim, (x3, y3), 1, (256, 0, 0), 2, cv2.LINE_AA)
            cv2.circle(tim, (x4, y4), 1, (256, 0, 0), 2, cv2.LINE_AA)
#             print("Pred")
            
            crop_img = img[min(y1, y2):max(y3, y4), min(x1, x3):max(x2, x4)]
            cv2.imwrite(os.path.join(path, dataset_4, "preprocess", "crop", folder_name,
                                     emotion, img_name), 
                        crop_img)
            
            # dewarp stage
            x1, y1 = preds[48].astype(np.int)
            x2, y2 = preds[105].astype(np.int)
            x3, y3 = preds[5].astype(np.int)
            x4, y4 = preds[21].astype(np.int)
            
            points = test_points([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], img.shape)
            
            x1, y1 = points[0]
            x2, y2 = points[1]
            x3, y3 = points[2]
            x4, y4 = points[3]
            
            cv2.circle(tim, (x1, y1), 1, (0, 0, 256), 2, cv2.LINE_AA)
            cv2.circle(tim, (x2, y2), 1, (0, 0, 256), 2, cv2.LINE_AA)
            cv2.circle(tim, (x3, y3), 1, (0, 0, 256), 2, cv2.LINE_AA)
            cv2.circle(tim, (x4, y4), 1, (0, 0, 256), 2, cv2.LINE_AA)
#             print("Pred")
            
            dewarp_img = four_point_transform(img,np.array([[x1, y1], 
                                                            [x2, y2], 
                                                            [x3, y3], 
                                                            [x4, y4]]))
            cv2.imwrite(os.path.join(path, dataset_4, "preprocess", "dewarp", folder_name,
                                     emotion, img_name), 
                        dewarp_img)
            
            
            # for pred in preds:
            pred = np.round(preds).astype(np.int)
            for i in range(pred.shape[0]):
                p = tuple(pred[i])
                cv2.circle(tim, p, 1, color, 1, cv2.LINE_AA)
            cv2.imwrite(os.path.join(path, dataset_4, "preprocess", "landmarks", folder_name,
                                     emotion, img_name), 
                        tim)
            

  0%|          | 599/134415 [00:10<32:16, 69.11it/s]  

--------SKIP--------


  1%|          | 685/134415 [00:11<31:29, 70.78it/s]

--------SKIP--------


  1%|          | 1035/134415 [00:16<31:57, 69.54it/s]

...

 94%|█████████▍| 471/500 [00:07<00:00, 67.01it/s]

--------SKIP--------


 85%|████████▍ | 423/500 [00:06<00:01, 69.10it/s]

--------SKIP--------


100%|██████████| 500/500 [00:07<00:00, 65.17it/s]
